In [1]:
a = [{'time': 0.78, 'rms_db': -61.46, 'pitch_hz': 798.76}, {'time': 0.81, 'rms_db': -61.64, 'pitch_hz': 579.98}, {'time': 0.84, 'rms_db': -62.13, 'pitch_hz': 579.71}, {'time': 0.87, 'rms_db': -63.45, 'pitch_hz': 471.93}]

In [6]:
for i in range(len(a)):
    # logging.info(f"pitch rms list of: {i}")
    # current_start = float(a[i]["time"])
    # next_start = float(a[i+1]["time"])
    # print("------------")

    # print("c",current_start)
    # print("n", next_start)
    # # print("s", start)
    print(a[i:3])

[{'time': 0.78, 'rms_db': -61.46, 'pitch_hz': 798.76}, {'time': 0.81, 'rms_db': -61.64, 'pitch_hz': 579.98}, {'time': 0.84, 'rms_db': -62.13, 'pitch_hz': 579.71}]
[{'time': 0.81, 'rms_db': -61.64, 'pitch_hz': 579.98}, {'time': 0.84, 'rms_db': -62.13, 'pitch_hz': 579.71}]
[{'time': 0.84, 'rms_db': -62.13, 'pitch_hz': 579.71}]
[]


In [2]:
import openai
import os
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv

In [5]:
load_dotenv()

openai.api_key = os.getenv("OPENAI_KEY")

loader = PyPDFLoader("data/150-WBE_500-TDS.pdf")
documents = loader.load()
print(len(documents))
# print(documents[0].text)

2


In [7]:
documents[0].page_content

'GENERAL PRODUCT INFORMATION\nColors: All standard and custom colors\nSolids Volume: 40%\nV.O.C.: 1.32 lbs per gallon catalyzed\nPot-life: 1-hour @ 70° F\nCure Schedule: 70° F @ 50% R.H. \n To touch: 6- Hours\n To re-coat: 8-10 Hours Minimum\n18-24 Hours Maximum\n Foot Traffic: 18-Hours\n Heavy Traffic: 72-Hours\nReducer: Not recommended\n Application Temp: 60ºF(15.6ºC)  - 90ºF(32.2ºC)\nEnvironment: For Interior Use Only  \nShelf Life: 12-months factory sealed  \nGENERAL PRODUCT PERFORMANCE\nTEST TYPE TEST METHOD RESULT\nChemical Resistance 50 MEK         \nDouble Rubs\n> 70% Gloss\nRetention\nHardness ASTMD 3363 HB\nAbrasion Resistance ASTMD-4060 40 mg loss\nFlexibility \n1/4” cylindrical mandrel ASTMD 522 Pass\nImpact Resistance ASTMD 2794 105 lb. direct\nCoefficient of Friction ASTMD-2047 > 0.6  /  pass\nUSES:\n » Primer for Resinous Flooring\n » Industrial Flooring\n » Large square footage\n » Economical 2-Coat Floor\nCoating\nADVANTAGES:\n » Long  working time\n » Easy to use 1:2 

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splitted_docs = text_splitter.split_documents(documents)

texts = [doc.page_content for doc in splitted_docs]

In [9]:
texts

['GENERAL PRODUCT INFORMATION\nColors: All standard and custom colors\nSolids Volume: 40%\nV.O.C.: 1.32 lbs per gallon catalyzed\nPot-life: 1-hour @ 70° F\nCure Schedule: 70° F @ 50% R.H. \n To touch: 6- Hours\n To re-coat: 8-10 Hours Minimum\n18-24 Hours Maximum\n Foot Traffic: 18-Hours\n Heavy Traffic: 72-Hours\nReducer: Not recommended\n Application Temp: 60ºF(15.6ºC)  - 90ºF(32.2ºC)\nEnvironment: For Interior Use Only  \nShelf Life: 12-months factory sealed  \nGENERAL PRODUCT PERFORMANCE',
 'GENERAL PRODUCT PERFORMANCE\nTEST TYPE TEST METHOD RESULT\nChemical Resistance 50 MEK         \nDouble Rubs\n> 70% Gloss\nRetention\nHardness ASTMD 3363 HB\nAbrasion Resistance ASTMD-4060 40 mg loss\nFlexibility \n1/4” cylindrical mandrel ASTMD 522 Pass\nImpact Resistance ASTMD 2794 105 lb. direct\nCoefficient of Friction ASTMD-2047 > 0.6  /  pass\nUSES:\n » Primer for Resinous Flooring\n » Industrial Flooring\n » Large square footage\n » Economical 2-Coat Floor\nCoating\nADVANTAGES:\n » Long  

In [11]:
# Generate embeddings for the texts
embeddings_model = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_KEY"))
embeddings = embeddings_model.embed_documents(texts)


In [12]:
url = "http://localhost:6333"
qdrant = Qdrant.from_texts(
    texts=texts,
    embedding=embeddings_model,
    url=url,
    prefer_grpc=False,
    collection_name="vector_db"
)

print("Vector DB Successfully Created!")

Vector DB Successfully Created!


In [13]:
import openai
import os
from qdrant_client import QdrantClient
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from dotenv import load_dotenv

In [14]:
load_dotenv()

openai.api_key = os.getenv("OPENAI_KEY")

url = "http://localhost:6333"

client = QdrantClient(
    url=url, prefer_grpc=False
)

In [15]:
print("##############")

collection_name = "vector_db"
embeddings_model = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_KEY"))
db = Qdrant(client=client, embeddings=embeddings_model, collection_name="vector_db")

llm = OpenAI(openai_api_key=os.getenv("OPENAI_KEY"))


##############


C:\Users\hp\AppData\Local\Temp\ipykernel_10436\1957597326.py:5: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  db = Qdrant(client=client, embeddings=embeddings_model, collection_name="vector_db")
C:\Users\hp\AppData\Local\Temp\ipykernel_10436\1957597326.py:7: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(openai_api_key=os.getenv("OPENAI_KEY"))


In [16]:
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)

def ask_question(question):
    result = retrieval_qa({"query": question})
    return result['result']

In [19]:
question = "could you please tell about maintainence"
answer = ask_question(question)
print("Question:", question)
print("Answer:", answer)

Question: could you please tell about maintainence
Answer:  According to the given context, maintenance for wear surface products involves frequent and thorough cleaning with a neutral PH cleaner, and the frequency of washing will vary depending on usage type, traffic, and age. It is also important to note that the product should not be applied to new slabs < 28-days old or concrete < 3500 PSI compression strength, and the ambient temperature should be between 60ºF and 90ºF with a relative humidity below 85%. It is not recommended for immersion service and should not be applied when the dew point is within 5ºF of the ambient temperature. Mock-ups and field test areas are recommended for validation of performance and appearance, and it is important to thoroughly read all Material Safety Data Sheets prior to use. Surface preparation is also crucial, and mechanical grinding or blasting is recommended to achieve the appropriate surface profile.
